In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("Datenbanken in Spark").getOrCreate()

In [2]:
spark

In [4]:
# inhalt eines directories anschauen
import os

for subdir, dirs, files in os.walk('./'):
    for file in files:
        print(file)

.profile
.bash_logout
.bashrc
Spark Intro.ipynb
.wget-hsts
a19537d3cf37c122db841d6fe4cd322bc10d1a558bb00d146b85cb9a
931dd2884a0ac1b9cbc2ea92a8a43452380620da30c9f613220f7e78
e7fd425a762b70d9075d0a1d715e0d6fbc7c3971fde1a1eec050a37d
dda1dc9c2defe69bd4d08f9ea4d3e75ee224c1c675a9b09b6d263c0d
31dcde5c4327f7fbc7fcd54fe164637e70d0bd85e0a9c23e4453403d
pyspark-3.3.1-py2.py3-none-any.whl
origin.json
ab2bb442149194bcbab5700d8f932112632ba9320db09910f4a4422b
fontlist-v330.json
jupyter_notebook_config.py
migrated
Spark Intro-checkpoint.ipynb
nbsignatures.db
notebook_secret
kernel-4f8a2bbe-dce9-4d0d-a287-2da7b2848b25.json
notebook_cookie_secret
nbserver-7-open.html
nbserver-7.json
history.sqlite
history.sqlite-journal
README
environments.txt
2022-12-05T03_08_27_542Z-debug-0.log


In [5]:
!pwd

/home/jovyan


In [6]:
!ls

'Spark Intro.ipynb'   studibier.csv   work


In [7]:
textfile = spark.read.text("Spark Intro.ipynb")

In [8]:
textfile.count()

177

In [9]:
textfile.first()

Row(value='{')

In [10]:
textfile.show()

+--------------------+
|               value|
+--------------------+
|                   {|
|          "cells": [|
|                   {|
|   "cell_type": "...|
|   "execution_cou...|
|   "id": "2bb33ef6",|
|     "metadata": {},|
|      "outputs": [],|
|         "source": [|
|    "from pyspark...|
|               "\n",|
|    "spark = Spar...|
|                   ]|
|                  },|
|                   {|
|   "cell_type": "...|
|   "execution_cou...|
|   "id": "dca54890",|
|     "metadata": {},|
|        "outputs": [|
+--------------------+
only showing top 20 rows



csv in spark einlesen

In [11]:
!cat studibier.csv

Name, Lieblingsbier, Alter
James, Weizen, 23
Money Penny, Brut IPA, 24
Lara, NEIPA, 22
Finn, Chimais, 17
Peter, NEIPA, 16
Jane, NEIPA, 22


In [12]:
studibier = spark.read.csv("studibier.csv")

In [13]:
studibier.show()

+-----------+--------------+------+
|        _c0|           _c1|   _c2|
+-----------+--------------+------+
|       Name| Lieblingsbier| Alter|
|      James|        Weizen|    23|
|Money Penny|      Brut IPA|    24|
|       Lara|         NEIPA|    22|
|       Finn|       Chimais|    17|
|      Peter|         NEIPA|    16|
|       Jane|         NEIPA|    22|
+-----------+--------------+------+



In [14]:
studibier.count()

7

In [16]:
studibier2 = spark.read.option("header", "true").csv("studibier.csv")

In [17]:
studibier2.show()

+-----------+--------------+------+
|       Name| Lieblingsbier| Alter|
+-----------+--------------+------+
|      James|        Weizen|    23|
|Money Penny|      Brut IPA|    24|
|       Lara|         NEIPA|    22|
|       Finn|       Chimais|    17|
|      Peter|         NEIPA|    16|
|       Jane|         NEIPA|    22|
+-----------+--------------+------+



In [18]:
studibier2

DataFrame[Name: string,  Lieblingsbier: string,  Alter: string]

In [19]:
studibier2.dtypes

[('Name', 'string'), (' Lieblingsbier', 'string'), (' Alter', 'string')]

In [20]:
studibier2.describe()

DataFrame[summary: string, Name: string,  Lieblingsbier: string,  Alter: string]

In [21]:
studibier = spark.read.option("header", "true").csv("studibier.csv", inferSchema=True)

In [22]:
studibier

DataFrame[Name: string,  Lieblingsbier: string,  Alter: double]

In [23]:
studibier.show()

+-----------+--------------+------+
|       Name| Lieblingsbier| Alter|
+-----------+--------------+------+
|      James|        Weizen|  23.0|
|Money Penny|      Brut IPA|  24.0|
|       Lara|         NEIPA|  22.0|
|       Finn|       Chimais|  17.0|
|      Peter|         NEIPA|  16.0|
|       Jane|         NEIPA|  22.0|
+-----------+--------------+------+



In [24]:
studibier.columns

['Name', ' Lieblingsbier', ' Alter']

In [32]:
studibier.select(" Lieblingsbier").distinct().show()

+--------------+
| Lieblingsbier|
+--------------+
|       Chimais|
|      Brut IPA|
|         NEIPA|
|        Weizen|
+--------------+



In [34]:
studibier.withColumnRenamed(' Alter', 'Alter').agg({"Alter": "avg"}).show()

+------------------+
|        avg(Alter)|
+------------------+
|20.666666666666668|
+------------------+



In [36]:
studibier.select([" Lieblingsbier", " Alter"]).distinct().show()

+--------------+------+
| Lieblingsbier| Alter|
+--------------+------+
|         NEIPA|  22.0|
|         NEIPA|  16.0|
|        Weizen|  23.0|
|      Brut IPA|  24.0|
|       Chimais|  17.0|
+--------------+------+



In [37]:
studibier.select([" Lieblingsbier", " Alter"]).show()

+--------------+------+
| Lieblingsbier| Alter|
+--------------+------+
|        Weizen|  23.0|
|      Brut IPA|  24.0|
|         NEIPA|  22.0|
|       Chimais|  17.0|
|         NEIPA|  16.0|
|         NEIPA|  22.0|
+--------------+------+



In [39]:
studibier.withColumn('Alter nach 2 Jahren', studibier[' Alter']+2).show()

+-----------+--------------+------+-------------------+
|       Name| Lieblingsbier| Alter|Alter nach 2 Jahren|
+-----------+--------------+------+-------------------+
|      James|        Weizen|  23.0|               25.0|
|Money Penny|      Brut IPA|  24.0|               26.0|
|       Lara|         NEIPA|  22.0|               24.0|
|       Finn|       Chimais|  17.0|               19.0|
|      Peter|         NEIPA|  16.0|               18.0|
|       Jane|         NEIPA|  22.0|               24.0|
+-----------+--------------+------+-------------------+



In [40]:
from pyspark.sql.functions import lit
studibier.withColumn('Alter nach 2 Jahren', studibier[' Alter']+2).withColumn('Eins', lit(1)).show()

+-----------+--------------+------+-------------------+----+
|       Name| Lieblingsbier| Alter|Alter nach 2 Jahren|Eins|
+-----------+--------------+------+-------------------+----+
|      James|        Weizen|  23.0|               25.0|   1|
|Money Penny|      Brut IPA|  24.0|               26.0|   1|
|       Lara|         NEIPA|  22.0|               24.0|   1|
|       Finn|       Chimais|  17.0|               19.0|   1|
|      Peter|         NEIPA|  16.0|               18.0|   1|
|       Jane|         NEIPA|  22.0|               24.0|   1|
+-----------+--------------+------+-------------------+----+



In [41]:
studibierMitSpalten = studibier.withColumn('Alter nach 2 Jahren', studibier[' Alter']+2).withColumn('Eins', lit(1))

In [42]:
studibierMitSpalten.drop('Eins').show()

+-----------+--------------+------+-------------------+
|       Name| Lieblingsbier| Alter|Alter nach 2 Jahren|
+-----------+--------------+------+-------------------+
|      James|        Weizen|  23.0|               25.0|
|Money Penny|      Brut IPA|  24.0|               26.0|
|       Lara|         NEIPA|  22.0|               24.0|
|       Finn|       Chimais|  17.0|               19.0|
|      Peter|         NEIPA|  16.0|               18.0|
|       Jane|         NEIPA|  22.0|               24.0|
+-----------+--------------+------+-------------------+



In [43]:
studibierMitSpalten.show()

+-----------+--------------+------+-------------------+----+
|       Name| Lieblingsbier| Alter|Alter nach 2 Jahren|Eins|
+-----------+--------------+------+-------------------+----+
|      James|        Weizen|  23.0|               25.0|   1|
|Money Penny|      Brut IPA|  24.0|               26.0|   1|
|       Lara|         NEIPA|  22.0|               24.0|   1|
|       Finn|       Chimais|  17.0|               19.0|   1|
|      Peter|         NEIPA|  16.0|               18.0|   1|
|       Jane|         NEIPA|  22.0|               24.0|   1|
+-----------+--------------+------+-------------------+----+



In [52]:
studibier.union(studibier).show()
studibier.schema

+-----------+--------------+------+
|       Name| Lieblingsbier| Alter|
+-----------+--------------+------+
|      James|        Weizen|  23.0|
|Money Penny|      Brut IPA|  24.0|
|       Lara|         NEIPA|  22.0|
|       Finn|       Chimais|  17.0|
|      Peter|         NEIPA|  16.0|
|       Jane|         NEIPA|  22.0|
|      James|        Weizen|  23.0|
|Money Penny|      Brut IPA|  24.0|
|       Lara|         NEIPA|  22.0|
|       Finn|       Chimais|  17.0|
|      Peter|         NEIPA|  16.0|
|       Jane|         NEIPA|  22.0|
+-----------+--------------+------+



StructType([StructField('Name', StringType(), True), StructField(' Lieblingsbier', StringType(), True), StructField(' Alter', DoubleType(), True)])

In [64]:
from pyspark.sql.types import StructType,StructField, StringType, DoubleType

newRows = spark.createDataFrame([("Max", "Weizen", None),(None, None, 20.0),("Tim", None, 21.0)],schema=studibier.schema)

naStudibier = studibier.union(newRows)
naStudibier.show()

+-----------+--------------+------+
|       Name| Lieblingsbier| Alter|
+-----------+--------------+------+
|      James|        Weizen|  23.0|
|Money Penny|      Brut IPA|  24.0|
|       Lara|         NEIPA|  22.0|
|       Finn|       Chimais|  17.0|
|      Peter|         NEIPA|  16.0|
|       Jane|         NEIPA|  22.0|
|        Max|        Weizen|  null|
|       null|          null|  20.0|
|        Tim|          null|  21.0|
+-----------+--------------+------+



In [65]:
naStudibier.na.drop(how="any").show()

+-----------+--------------+------+
|       Name| Lieblingsbier| Alter|
+-----------+--------------+------+
|      James|        Weizen|  23.0|
|Money Penny|      Brut IPA|  24.0|
|       Lara|         NEIPA|  22.0|
|       Finn|       Chimais|  17.0|
|      Peter|         NEIPA|  16.0|
|       Jane|         NEIPA|  22.0|
+-----------+--------------+------+

